In [79]:
import torch
from torch import utils
from torchvision import datasets, transforms

from torch.utils.data import DataLoader


import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [80]:
path='./MNIST_data'

#### Смотрим на данные

In [81]:
train_data = datasets.MNIST(path, train=True, download=True, transform=transforms.ToTensor())
train_data


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./MNIST_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [82]:
# параметры датасета
len_train = len(train_data)
print(f"кол-во элементов в датасете: {len_train}")
print(f"тип элемента:  {type(train_data[0])}")
print(f"длинна элемента:  {len(train_data[0])}")

кол-во элементов в датасете: 60000
тип элемента:  <class 'tuple'>
длинна элемента:  2


In [83]:
# посмотрим на содержимое 
item = train_data[0][0] 
print(item.shape)
print(f"min: {item.min()} max: {item.max()}")
#train_data[0][0]

torch.Size([1, 28, 28])
min: 0.0 max: 1.0


In [119]:
tensors = train_data.transform(train_data.data.numpy())
tensors.mean()
tensors.std()

tensor(0.3081)

#### Вычисления и преобразования

Судя по всему, при загрузке, по умолчанию была произведена min-max нормализация
Наша задача сделать std нормализацию 
Для этого найдем среднее и стандартное отклонение

In [132]:
def get_train_mean_std(train_set):
    """
    Получаем тренировочный датасет
    Возвращаем кортеж (mean, std) 
    ==============================
    
    Version 01:
    items_ = [train_set[i][0].numpy() for i in range(len(train_set))]
    mean_ = np.mean(items_)
    std_ = np.std(items_)
    return mean_, std_ 
    
    Version 02:
    # загружаем и приводим к тензору данные из тренировочного датасета
    mnist_dl = DataLoader(dataset=train_set, batch_size=len(train_set))
    items, _ = next(iter(mnist_dl))
    print(items.shape)

    """
   
    # обращаемся к сырым дынным и трансформируем их 
    items = train_set.transform(train_set.data.numpy())
    print(items.shape)
    return float(items.mean()), float(items.std())

In [133]:
# получим расчетные данные
k_mean, k_std = get_train_mean_std(train_data)
print(f"Mean = {k_mean}\nStd = {k_std}")

torch.Size([28, 60000, 28])
Mean = 0.13062907755374908
Std = 0.30810782313346863


In [134]:
mnist_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((k_mean,), (k_std,))
                ])

In [135]:

# загружаем с модифицированной нормализацией
train_data_mod = datasets.MNIST(path, train=True, download=True, transform=mnist_transform)

In [136]:
# проверяем результат
mod_mean, mod_std = get_train_mean_std(train_data_mod)
print(f"Mean = {np.round(mod_mean,5)}\nStd = {np.round(mod_std, 5)}")

torch.Size([28, 60000, 28])
Mean = 0.00112
Std = 1.0


### OK!
